<a href="https://colab.research.google.com/github/mrkim21/mrkim21.github.io/blob/main/appfolder/appcodes/20240118_spelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌀 Spelling checker

[applink](https://mrkim21.github.io/appfolder/spelling.html)

In [ ]:
!pip install gradio gTTS

In [ ]:
import pandas as pd
import random
from gtts import gTTS
import gradio as gr

# Function to read the CSV and choose a random word
def choose_word():
    data = pd.read_csv('./wordlist.csv')
    chosen_row = random.choice(data.index)
    word_info = data.loc[chosen_row]
    return word_info

# Function to generate speech from the chosen word
def text_to_speech(word_info):
    text = f"This word is pronounced as {word_info['Words']}, and its part of speech is {word_info['POS']}. Here's a usage: {word_info['Usage']}. Write the spelling of this word."
    tts = gTTS(text, lang='en')
    tts.save('./word_audio.mp3')
    return './word_audio.mp3', word_info['Words']

# Function to check spelling
def check_spelling(user_spelling, correct_spelling):
    if user_spelling.strip().lower() == correct_spelling.lower():
        return "Correct!"
    else:
        return f"Incorrect! The correct spelling is {correct_spelling}."

# Initialize the app
def gradio_app():
    word_info = choose_word()
    audio_file, correct_spelling = text_to_speech(word_info)

    def update_audio_file(_):
        new_word_info = choose_word()
        new_audio_file, new_correct_spelling = text_to_speech(new_word_info)
        return new_audio_file, new_correct_spelling

    with gr.Blocks() as demo:
        audio = gr.Audio(value=audio_file, label="Listen to the word", interactive=False)
        spelling_input = gr.Textbox(label="Type the spelling here")
        submit_button = gr.Button("Submit")
        correct_label = gr.Label("")
        next_button = gr.Button("Next Word")
        state_correct_spelling = gr.State(value=correct_spelling)

        submit_button.click(check_spelling, inputs=[spelling_input, state_correct_spelling], outputs=correct_label)
        next_button.click(update_audio_file, inputs=[], outputs=[audio, state_correct_spelling])

    return demo

# Run the app
gradio_app().launch()
